In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import tensorflow as tf
import seaborn as sns
from tqdm import tqdm
import warnings

pd.set_option('display.max_columns', None)
# 모든 columns를 생략하지 말고 다 보여준다.
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/test.csv')
origin = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/jm1.csv')

In [ ]:
columns = ['uniq_Op','uniq_Opnd','total_Op','total_Opnd','branchCount']

for col in columns:
    origin[col][origin[col] == '?'] = np.nan
    origin[col] = origin[col].astype(float)


<ipython-input-3-d9b32314e3b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  origin[col][origin[col] == '?'] = np.nan


In [ ]:
train_data = pd.concat([train_data,origin],ignore_index=True)
train_data = train_data.drop_duplicates()

In [ ]:
train_data

,id,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,b,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
0,0.0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,5448.79,0.09,302.71,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,1.0,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,936.71,0.05,52.04,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,2.0,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,1754.01,0.07,97.45,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
3,3.0,8.0,1.0,1.0,1.0,23.0,94.01,0.19,5.25,17.86,473.66,0.03,26.31,4,0,2,0,8.0,6.0,16.0,7.0,1.0,True
4,4.0,11.0,2.0,1.0,2.0,17.0,60.94,0.18,5.63,12.44,365.67,0.02,20.31,7,0,2,0,7.0,6.0,10.0,10.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112643,NaN,18.0,4.0,1.0,4.0,52.0,241.48,0.14,7.33,32.93,1770.86,0.08,98.38,13,0,2,0,10.0,15.0,30.0,22.0,7.0,False
112644,NaN,9.0,2.0,1.0,2.0,30.0,129.66,0.12,8.25,15.72,1069.68,0.04,59.43,5,0,2,0,12.0,8.0,19.0,11.0,3.0,False
112645,NaN,42.0,4.0,1.0,2.0,103.0,519.57,0.04,26.40,19.68,13716.72,0.17,762.04,29,1,10,0,18.0,15.0,59.0,44.0,7.0,False
112646,NaN,10.0,1.0,1.0,1.0,36.0,147.15,0.12,8.44,17.44,1241.57,0.05,68.98,6,0,2,0,9.0,8.0,21.0,15.0,1.0,False


In [ ]:
for col in train_data.columns:
    plt.figure(figsize=(18,8))
    ax1 = plt.subplot(1,2,1)
    sns.boxplot(data=train_data,x='defects',y=col)
    plt.title('defects -- ' + col + 'train_data')
    ax2 = plt.subplot(1,2,2)
    sns.boxplot(data=test_data,y=col)
    plt.title(col + '/ test_data')
    plt.show()


In [ ]:
train_data_clean = train_data.drop(train_data[train_data['loc'] > 3000].index)
train_data_clean = train_data.drop(train_data[train_data['l'] > 1.2].index)
train_data_clean = train_data.drop(train_data[train_data['e'] > 30000000].index)
train_data_clean = train_data.drop(train_data[train_data['b'] > 25].index)
train_data_clean = train_data.drop(train_data[train_data['t'] > 1500000].index)
train_data_clean = train_data.drop(train_data[train_data['lOCode'] > 2500].index)
train_data_clean = train_data.drop(train_data[train_data['locCodeAndComment'] > 100].index)
train_data_clean = train_data.drop(train_data[train_data['total_Op'] > 5000].index)
train_data_clean = train_data.drop(train_data[train_data['total_Opnd'] > 3000].index)


In [ ]:
columns = ['loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e', 'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount']
for col in columns:
    print(train_data_clean[col].isnull().sum(),f'{col}의 null 개수 ')

In [ ]:
train_data_clean.drop(['id'],axis=1,inplace=True)
train_data_clean


In [ ]:
train_data_clean.dropna(axis=0,inplace=True)

In [ ]:
for col in columns:
    print(train_data_clean[col].isnull().sum(),f'{col}의 null 개수 ')

0 loc의 null 개수 
0 v(g)의 null 개수 
0 ev(g)의 null 개수 
0 iv(g)의 null 개수 
0 n의 null 개수 
0 v의 null 개수 
0 l의 null 개수 
0 d의 null 개수 
0 i의 null 개수 
0 e의 null 개수 
0 b의 null 개수 
0 t의 null 개수 
0 lOCode의 null 개수 
0 lOComment의 null 개수 
0 lOBlank의 null 개수 
0 locCodeAndComment의 null 개수 
0 uniq_Op의 null 개수 
0 uniq_Opnd의 null 개수 
0 total_Op의 null 개수 
0 total_Opnd의 null 개수 
0 branchCount의 null 개수 


In [ ]:
train_data_clean['defects'] = train_data_clean['defects'].astype(float)

In [ ]:
X_train = train_data_clean.drop(['defects'],axis=1)
Y_train = train_data_clean['defects']

In [ ]:
X_test = test_data.drop('id',axis=1)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
def contains_bool_values(arr):
    """
    넘파이 배열에서 bool값이 들어있는지 확인하는 함수입니다.

    Args:
        arr: 넘파이 배열

    Returns:
        bool값이 들어있는지 여부
    """

    return any(isinstance(x, bool) for x in arr)

contains_bool_values(Y_train)

False

In [ ]:
skf = StratifiedKFold(n_splits = 5, shuffle=True,random_state=42)

models = []
auc_scores = []
test_predictions = np.zeros(len(X_test))

for fold,(train_index,val_index) in enumerate(skf.split(X_train,Y_train)):
    X_train_fold,X_val_fold = X_train[train_index],X_train[val_index]
    y_train_fold,y_val_fold = Y_train.iloc[train_index],Y_train.iloc[val_index]

    model = Sequential()
    model.add(Dense(128,input_shape = (X_train_fold.shape[1],),activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy',tf.keras.metrics.AUC()])
    early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights = True)
    class_weights = {0:1, 1:len(y_train_fold[y_train_fold==0]) / len(y_train_fold[y_train_fold == 1])}

    epochs = 100
    batch_size = 32
    progress_bar = tqdm(range(epochs),position=0,desc = 'Fold Progress')

    for epoch in progress_bar:
        history = model.fit(X_train_fold,y_train_fold,epochs=1,
                           batch_size=batch_size,class_weight=class_weights,
                           verbose=0,
                           validation_data=(X_val_fold,y_val_fold),
                           callbacks=[early_stopping])
        progress_bar.set_description(f'Epoch {epoch+1}/{epochs} - Loss: {history.history["loss"][0]:.4f} - Accuracy: {history.history["accuracy"][0]:.4f}')
        #progress_bar.set_description(f'Epoch {epoch + 1}/{epochs} - Loss: {history.history['loss'][0] : .4f} - Accuracy:{history.history['accuracy'][0] : .4f}')

    y_pred_prob = model.predict(X_val_fold)
    auc = roc_auc_score(y_val_fold,y_pred_prob)
    auc_scores.append(auc)

    test_predictions += model.predict(X_test).flatten()

    model_filename = f'/kaggle/working/model_fold_{fold}.h5'
    model.save(model_filename)
    models.append(model_filename)

average_auc = sum(auc_scores) / len(auc_scores)
print(f'Average AUC score : {average_auc:.4f}')

test_predictions /= skf.n_splits

print('Saved models:')
print(models)



Epoch 100/100 - Loss: 0.8597 - Accuracy: 0.7480: 100%|██████████| 100/100 [08:09<00:00,  4.90s/it]

102/692 [===>..........................] - ETA: 0s

2121/2121 [==============================] - 2s 863us/step


Epoch 100/100 - Loss: 0.8586 - Accuracy: 0.7498: 100%|██████████| 100/100 [08:05<00:00,  4.85s/it]

130/692 [====>.........................] - ETA: 0s

...

113/692 [===>..........................] - ETA: 0s

2121/2121 [==============================] - 2s 816us/step


Epoch 100/100 - Loss: 0.8603 - Accuracy: 0.7517: 100%|██████████| 100/100 [07:56<00:00,  4.76s/it]

117/692 [====>.........................] - ETA: 0s

2121/2121 [==============================] - 2s 827us/step
Average AUC score : 0.7809
Saved models:
['/kaggle/working/model_fold_0.h5', '/kaggle/working/model_fold_1.h5', '/kaggle/working/model_fold_2.h5', '/kaggle/working/model_fold_3.h5', '/kaggle/working/model_fold_4.h5']


### 1. Y_val에 input contains NaN 에러 발견  -> null값 행을 제거
### 2. model 층을 더 깊게 만들어서 두 번쨰 제출
### 3. batchsize를 좀 더 키워보는게 loss를 줄이는데 좋다고 함

In [ ]:


submission_df = pd.read_csv('/content/drive/MyDrive/캐글 software defects prediction/sample_submission.csv')
submission_df.defects = test_predictions.reshape(-1)
submission_df.to_csv('/content/drive/MyDrive/캐글 software defects prediction/sample_submission.csv',index=False)
submission_df.head()

,id,defects
0,101763,0.532454
1,101764,0.442405
2,101765,0.836313
3,101766,0.756434
4,101767,0.400602


In [ ]:
submission_df

,id,defects
0,101763,0.508107
1,101764,0.487201
2,101765,0.851189
3,101766,0.754795
4,101767,0.381938
...,...,...
67837,169600,0.592404
67838,169601,0.358971
67839,169602,0.458813
67840,169603,0.268308


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
